In [1]:
from qdrant_client import QdrantClient

In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True